In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit

data = pd.read_csv('TRAIN_FINAL.csv')
data.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_360,col_361,col_362,col_363,col_364,col_365,col_366,col_367,col_368,TARGET
0,2,23,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56264.010000,0
1,2,51,510.0,897.72,2393.58,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
2,2,23,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83213.610000,0
3,2,23,120.0,59.67,59.67,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75297.300000,0
4,2,38,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176487.690000,0


In [25]:
from xgboost import XGBClassifier
classifiers = []
n = 10

no_frauds = len(data[data['TARGET'] == 1])
non_fraud_indices = data[data.TARGET == 0].index
for i in range(n) :
    random_indices = np.random.choice(non_fraud_indices,no_frauds, replace=False)
    fraud_indices = data[data.TARGET == 1].index
    under_sample_indices = np.concatenate([fraud_indices,random_indices])
    under_sample = data.loc[under_sample_indices]
    X_train = under_sample.loc[:,under_sample.columns != 'TARGET']
    y_train = under_sample.loc[:,under_sample.columns == 'TARGET']
    clf = XGBClassifier()
    clf.fit(X_train,y_train)
    classifiers.append(clf)

In [26]:
df_test = pd.read_csv("Test_v1.csv")

data_out = df_test
df_test = df_test.drop(['index'], axis=1)

In [27]:
y_predictions = []
for clsf in classifiers :
    dt_y_pred = clsf.predict(df_test)
    y_predictions.append(dt_y_pred)

/home/ssk/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ssk/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ssk/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ssk/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

In [23]:
# print(type(dt_y_pred))
# final = numpy.zeros(len(data_out.index))
# print(type(final[1]))

In [28]:
import numpy
final_pred = numpy.zeros(len(data_out.index))
for i in range(len(data_out.index)) :
    sum = 0
    for y_pred in y_predictions :
        if y_pred[i] == 1 :
            sum += 1
    if sum >= n/2 :
        final_pred[i] = 1

final_pred

array([0., 0., 0., ..., 0., 0., 1.])

In [29]:
data_op = data_out
data_op["TARGET"] = final_pred
df_submission = pd.DataFrame({'index':data_out.index, 'TARGET':data_op.TARGET})
df_submission.to_csv("project2_1.csv",index=False)